In [1]:
import os
os.environ['TF_KERAS'] = '1'
import onnx
import numpy as np
import re
from collections import Counter
import tensorflow as tf
import glob
from nilearn.input_data import NiftiMasker
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from keras.layers import TimeDistributed
from nilearn.masking import apply_mask, unmask
from nilearn.image import resample_to_img
from nilearn.input_data import NiftiMasker
from nilearn.masking import compute_epi_mask
from nibabel.testing import data_path
import nibabel as nib
from keras.layers import LSTM
import nilearn
import argparse
import keras
import keras.backend
import keras.layers
import keras.models
import keras.utils
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.layers import Conv2D,Dropout,Flatten,Activation,MaxPool2D
from keras.optimizers import Adam
import os
#from tensorflow.keras.models import Sequential
from sklearn.model_selection import LeaveOneGroupOut
import nipype.interfaces.io as nio
import matplotlib
import matplotlib.pyplot as plt

from keras.utils import np_utils
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from tqdm import tqdm
import scipy.io as sio
from nilearn.image import index_img
from nilearn import image
from nilearn import plotting
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from nilearn.input_data import NiftiLabelsMasker

ModuleNotFoundError: No module named 'onnx'

# Importing Dataset


In [2]:
#getting labels 
phenotypic = pd.read_csv('/Users/pranjal27bhardwaj/Desktop/allSubs_testSet_phenotypic_dx.csv')
labels = phenotypic['DX']
labels = pd.DataFrame(labels)
ID = phenotypic['ScanDir ID']
ID = pd.DataFrame(ID)
result = pd.concat([ID,labels], axis=1, join='inner')

In [3]:
#getting fmri image path of one website
nifti_files = glob.glob('/Users/desktop/fmri_KKI/*')

In [4]:
#separating Id from the list of path 
s_nifti = []
for i in nifti_files:
    s_nifti.append(re.split('(\d+)',i))

In [5]:
#separting ID from the list of path 
s1_nifti = []
for i in range(len(s_nifti)):
    s1_nifti.append(s_nifti[i][1])

In [ ]:
#calculating the voxels
import numpy as np
import nibabel as nb

# Load data
nii = nb.load(nifti_files[0])
img = nii.get_fdata()

# Get voxel dimensions
voxel_dims = (nii.header["pixdim"])[1:4]
print("Voxel dimensions:")
print("  x = {} mm".format(voxel_dims[0]))
print("  y = {} mm".format(voxel_dims[1]))
print("  z = {} mm".format(voxel_dims[2]))

# Compute volume
nonzero_voxel_count = np.count_nonzero(img)
voxel_volume = np.prod(voxel_dims)
nonzero_voxel_volume = nonzero_voxel_count * voxel_volume

print("Number of non-zero voxels = {}".format(nonzero_voxel_count))
print("Volume of non-zero voxels = {} mm^3".format(nonzero_voxel_volume))


In [ ]:
epi_img = index_img(nifti_files[0], slice(0, 100))
masker = NiftiMasker(mask_strategy='epi')
s1 = masker.fit(epi_img)
type(s1)

In [ ]:
fmri_masked.shape

In [6]:
## sorting labels according to file

Id = result['ScanDir ID'].values.tolist()
label = result['DX'].values.tolist()
#df2 = s1_nifti[0].values.tolist() 

df_sort = []
for i in range(len(s1_nifti)):
    df_sort.append(int(s1_nifti[i]))

In [7]:
#sorting the labels according to file

new_list = []
for i in range(len(df_sort)):
    if df_sort[i] in Id:
        s = Id.index(df_sort[i])
        #print(df_sort[i])
        new_list.append(label[s])
        #print(label[s])

In [8]:
Img = nib.load('/Users/vidhi/Desktop/KKI_data/' + s1_nifti[2] +'.nii.gz')
x = Img.get_data()
x.shape

/Users/vidhi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  


(53, 64, 46)

In [9]:
#performing 2mm smoothing on whole data
#reshaping the dimensions of the data before passing to  cnn
Dataset = []
for i in range(82):
    Img = nib.load('/Users/vidhi/Desktop/KKI_data/' + s1_nifti[i] +'.nii.gz')
    x = Img.get_data()
    #result = x[:, :, :,:1]
    Dataset.append(x)

/Users/vidhi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  


In [10]:
X = np.array(Dataset)
Y = np.array(new_list)

In [ ]:
#Binary classification
for i in range(len(new_list)):
    if new_list[i] > 1:
        new_list[i] = 1
    else:
        continue 
        
#X = np.array([Dataset], dtype = object)
#Y = np.array(new_list)

In [11]:
Counter(Y)

Counter({0: 60, 1: 16, 2: 1, 3: 5})

In [12]:
X.shape

(82, 53, 64, 46)

# OHSU dataset

In [13]:
#getting labels 
phenotypic_ohsu = pd.read_csv('/Users/vidhi/Desktop/OHSU_phenotypic.csv')
labels_ohsu = phenotypic_ohsu['DX']
labels_ohsu = pd.DataFrame(labels_ohsu)
ID_ohsu = phenotypic_ohsu['ScanDir ID']
ID_ohsu = pd.DataFrame(ID_ohsu)
result_ohsu = pd.concat([ID_ohsu,labels_ohsu], axis=1, join='inner')

In [14]:
ohsu_files = glob.glob('/Users/vidhi/Desktop/fmri_ohsu 2/*')

In [15]:
img_path = [x for x in ohsu_files if re.search('run1', x)]
ohsu = []
for i in img_path:
    ohsu.append(re.split('(\d+)',i))
ohsu_id = []
for i in range(len(ohsu)):
    ohsu_id.append(ohsu[i][3])

In [16]:
## sorting labels according to file

Id_ohsu = result_ohsu['ScanDir ID'].values.tolist()
label_ohsu = result_ohsu['DX'].values.tolist()
#df2 = s1_nifti[0].values.tolist() 

id_int = []
for i in range(len(ohsu_id)):
    id_int.append(int(ohsu_id[i]))

In [17]:
#sorting the labels according to file
sorted_labels = []
for i in range(len(id_int)):
    if id_int[i] in Id_ohsu:
        s1 = Id_ohsu.index(id_int[i])
        #print(id_int[i])
        sorted_labels.append(label_ohsu[s1])
        #print(label_ohsu[s1])

In [18]:
#Performing smoothing and cleaning of image on whole dataset and saving it to another folder
ohsu_data = []
for i in range(len(img_path)):
    smoothing = image.smooth_img(img_path[i], 2)
    new_img = image.clean_img(smoothing, sessions=None, detrend=True ,standardize= True)
    s1 = image.index_img(new_img,10)
    x1 = s1.get_data()
    ohsu_data.append(x1)
    
    #nib.save(s2 , '/Users/vidhi/Desktop/KKI_data/' + s1_nifti[i] +'.nii.gz')

/Users/vidhi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys


In [19]:
X_ohsu = np.array(ohsu_data)
Y_ohsu = np.array(sorted_labels)

In [20]:
X_ohsu = np.array(ohsu_data)
Y_ohsu = np.array(sorted_labels)

In [21]:
#counting the labels of 
Counter(Y_ohsu)

Counter({1: 30, 0: 42, 3: 5, 2: 2})

In [22]:
#combining both dataset
x_combined = np.concatenate((X, X_ohsu))
y_combined = np.concatenate((Y,Y_ohsu))

In [23]:
#using counter to check if data is balanced or not
Counter(y_combined)

Counter({0: 102, 1: 46, 2: 3, 3: 10})

In [24]:
#reshaping the train data value
reshaped_data = []
for i in range(len(x_combined)):
    reshaped_data.append(x_combined[i].reshape(1,53,64,46))
X_data = np.array(reshaped_data)

In [27]:
#converting to categorical
Y_new = keras.utils.to_categorical(y_combined, 4)


In [30]:
#splitting the dataset
x_train, x_test, y_train, y_test = train_test_split(X_data,Y_new,shuffle = True, random_state = 32, test_size=0.15)

In [33]:
#model making
model = keras.models.Sequential() 
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(32, kernel_size= 3, kernel_initializer='uniform', padding='same'),
                                              input_shape=(None,53,64,46)))

model.add(keras.layers.TimeDistributed(keras.layers.Activation("relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPool2D(pool_size=2)))
#model.add(keras.layers.TimeDistributed(keras.layers.Dropout(0.2)))

model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(32, kernel_size= 3, kernel_initializer='uniform', padding='same',
                              activation = 'relu')))
model.add(TimeDistributed(keras.layers.MaxPool2D(pool_size=2)))


model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(32, kernel_size= 3, kernel_initializer='uniform', padding='same',
                              activation='relu')))
model.add(TimeDistributed(keras.layers.MaxPool2D(pool_size = 2)))


model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(32, kernel_size= 3, kernel_initializer='uniform', padding='same',
                              activation='relu')))
model.add(TimeDistributed(keras.layers.MaxPool2D(pool_size = 2)))


model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(32, kernel_size= 3, kernel_initializer='uniform', padding='same',
                              activation='relu')))
#model.add(TimeDistributed(keras.layers.MaxPool2D(pool_size = 2)))

model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(32, kernel_size= 3, kernel_initializer='uniform', padding='same',
                              activation='relu')))
#model.add(TimeDistributed(keras.layers.MaxPool2D(pool_size=2)))

model.add(keras.layers.TimeDistributed(keras.layers.Dropout(0.2)))
model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))

model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100))

model.add(keras.layers.Dense(250, kernel_initializer='he_normal',activation = 'relu'))
model.add(keras.layers.Dense(250, kernel_initializer='he_normal',activation = 'relu'))
#model.add(keras.layers.Dense(512, kernel_initializer='he_normal',activation = 'relu'))
model.add(keras.layers.Dense(4))
model.add(keras.layers.Activation('softmax'))
#model.add(Activation('linear'))
    
#model.add(keras.layers.Activation('softmax'))
    
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.1)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#model.build(input_shape)
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_27 (TimeDis (None, None, 53, 64, 32)  13280     
_________________________________________________________________
time_distributed_28 (TimeDis (None, None, 53, 64, 32)  0         
_________________________________________________________________
time_distributed_29 (TimeDis (None, None, 26, 32, 32)  0         
_________________________________________________________________
time_distributed_30 (TimeDis (None, None, 26, 32, 32)  9248      
_________________________________________________________________
time_distributed_31 (TimeDis (None, None, 13, 16, 32)  0         
_________________________________________________________________
time_distributed_32 (TimeDis (None, None, 13, 16, 32)  9248      
_________________________________________________________________
time_distributed_33 (TimeDis (None, None, 6, 8, 32)    0         
__________

In [34]:
#model.fit(x_train, y_train,epochs= 200, verbose=1)

In [ ]:
model.evaluate(x_test,y_test)

In [35]:
#cross validation
Kfold = KFold(n_splits=7)
Kfold.get_n_splits(x_train, y_train)  
foldNum=0                                  #initializing fold = 0
for train_index, val_index in Kfold.split(x_train, y_train):
    foldNum+=1
    print("fold",foldNum)
    X_train, X_val = x_train[train_index], x_train[val_index]
    Y_train, Y_val = y_train[train_index], y_train[val_index]
    print(X_train.shape)
    #training the dataset
    history1 = model.fit(X_train, Y_train, validation_data = (X_val,Y_val), epochs=20,batch_size=32, verbose=1)

fold 1
(116, 1, 53, 64, 46)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 116 samples, validate on 20 samples
Epoch 1/20





116/116 [==============================] - 3s 24ms/step - loss: 1.3702 - acc: 0.5172 - val_loss: 1.3139 - val_acc: 0.7500
Epoch 2/20
116/116 [==============================] - 0s 4ms/step - loss: 1.2725 - acc: 0.6121 - val_loss: 1.0047 - val_acc: 0.7500
Epoch 3/20
116/116 [==============================] - 0s 4ms/step - loss: 0.9946 - acc: 0.6121 - val_loss: 0.6960 - val_acc: 0.7500
Epoch 4/20
116/116 [==============================] - 0s 4ms/step - loss: 0.9806 - acc: 0.6121 - val_loss: 0.7104 - val_acc: 0.7500
Epoch 5/20
116/116 [==============================] - 0s 4ms/step - loss: 0.9412 - acc: 0.6121 - val_loss: 0.7663 - val_acc: 0.7500
Epoch 6/20
116/116 [==============================] - 0s 4ms/step - loss: 0.9308 - acc: 0.6121 - val_loss: 0.7841 - val_acc: 0.7500
Epoch 7/20
116/116 [============

116/116 [==============================] - 0s 4ms/step - loss: 0.7990 - acc: 0.6724 - val_loss: 1.3215 - val_acc: 0.4000
Epoch 7/20
116/116 [==============================] - 1s 4ms/step - loss: 0.8017 - acc: 0.6724 - val_loss: 1.3054 - val_acc: 0.4000
Epoch 8/20
116/116 [==============================] - 1s 4ms/step - loss: 0.8003 - acc: 0.6724 - val_loss: 1.3102 - val_acc: 0.4000
Epoch 9/20
116/116 [==============================] - 0s 4ms/step - loss: 0.7943 - acc: 0.6724 - val_loss: 1.3063 - val_acc: 0.4000
Epoch 10/20
116/116 [==============================] - 0s 4ms/step - loss: 0.7934 - acc: 0.6724 - val_loss: 1.3116 - val_acc: 0.4000
Epoch 11/20
116/116 [==============================] - 0s 4ms/step - loss: 0.7942 - acc: 0.6724 - val_loss: 1.3245 - val_acc: 0.4000
Epoch 12/20
116/116 [==============================] - 0s 4ms/step - loss: 0.7930 - acc: 0.6724 - val_loss: 1.3373 - val_acc: 0.4000
Epoch 13/20
116/116 [==============================] - 0s 4ms/step - loss: 0.7944 - 

In [36]:
model.evaluate(x_test,y_test)

25/25 [==============================] - 0s 2ms/step


[0.9176188111305237, 0.6399999856948853]

In [ ]:
#model.predict(x_test)

In [ ]:
model.save('FMRI_model.h5') 

In [ ]:
model.save_weights('FMRI_weights.h5')